In [5]:
import json
from utils.schema import *
from utils.reverse_logic import *
from utils.reverse_sql import *
from utils.reverse_middle import *
from utils.parse import *
import os
import random
import copy

In [6]:
with open('raw_data/tables.json','r') as f:
    db = json.load(f)
db_dict = {}
schemas, db_names, tables = get_schemas_from_json('raw_data/tables.json')
for item in db:
    db_dict[item['db_id']] = item

db_reverse = {}
db_c_ori2pre = {}
db_t_ori2pre = {}
for k,v in db_dict.items():
    db_reverse[k] = {}
    column = []
    table = []
    db_c_ori2pre[k] = {}
    db_t_ori2pre[k] = {}
    for o,c in zip(v['column_names'],v['column_names_original']):
        column.append([o[0],o[1],c[1]])
        db_c_ori2pre[k][c[1]] = o[1]
    db_reverse[k]['column'] = column
    for o,c in zip(v['table_names'],v['table_names_original']):
        table.append([o,c])
        db_t_ori2pre[k][c] = o
    db_reverse[k]['table'] = table
db_aug = {}
for k,v in db_dict.items():
    col_temp = {}
    for index, item in enumerate(v['column_names']):
        if item[0] not in col_temp.keys():
            if item[1] in v['primary_keys']:
                col_temp[item[0]] = [(index,item[1],v['column_types'][index],'primary')]
            else:
                col_temp[item[0]] = [(index,item[1],v['column_types'][index],'None')]
        else:
            if item[1] in v['primary_keys']:
                col_temp[item[0]].append((index,item[1],v['column_types'][index],'primary'))
            else:
                col_temp[item[0]].append((index,item[1],v['column_types'][index],'None'))
    db_aug[k] = col_temp

In [7]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
db_value = {}
for k,v in db_dict.items():
    col_value = {}
    db_id = v['db_id']
    db_file = os.path.join('raw_data/database', db_id, db_id + '.sqlite')
    if not os.path.exists(db_file):
        raise ValueError('[ERROR]: database file %s not found ...' % (db_file))
    conn = sqlite3.connect(db_file)
    conn.text_factory = lambda b: b.decode(errors='ignore')
    conn.execute('pragma foreign_keys=ON')
    for i, (tab_id, col_name) in enumerate(db_dict[db_id]['column_names_original']):
        if i == 0 : # ignore * and special token 'id'
            continue
        tab_name = db_dict[db_id]['table_names_original'][tab_id]
        cursor = conn.execute("SELECT DISTINCT \"%s\" FROM \"%s\";" % (col_name, tab_name))
        cell_values = cursor.fetchall()
        cell_values = [str(each[0]) for each in cell_values]
        cell_values = [[str(float(each))] if is_number(each) else each.lower().split() for each in cell_values]
        col_value[i] = cell_values
    db_value[k] = col_value

In [8]:
random.seed(999)
string_no_agg = ['min','max','avg','sum']
value_no_agg = ['count']
time_no_agg = ['avg','sum','count']
def replace_select(middle,db_id,num = 1,agg = [''],distinct = [False]):
    table_idxs = middle['from']['table']
    col_ori = []
    col_aug = db_aug[db_id]
    for item in middle['select']:
        col_ori.append((item[0],item[2]))
    replaced = []
    result = []
    step = 0
    for index in range(num):
        flag = True
        while flag:
            if step >= 20:
                return (middle ,replaced, False)
            tab_index = random.randint(0,len(table_idxs)-1)
            col_idxs = col_aug[table_idxs[tab_index]]
            col_index = random.randint(0,len(col_idxs)-1)
            if num > 1:
                if col_index == 0:
                    step += 1
                    continue
            if col_index == 0:
                distinct[index] = False
            if (col_idxs[col_index][0],agg[index]) in col_ori:
                step += 1
                continue
            if agg[index] != '':
                if agg[index] in string_no_agg and col_idxs[col_index][2] == 'text':
                    step += 1
                    continue
                if agg[index] in value_no_agg and col_idxs[col_index][2] == 'number' :
                    step += 1
                    continue
                if agg[index] in time_no_agg and col_idxs[col_index][2] == 'time' :
                    step += 1
                    continue
            flag = False
            result.append([col_idxs[col_index][0],table_idxs[tab_index],agg[index],distinct[index]])
            replaced.append(col_idxs[col_index][1])
            col_ori.append((col_idxs[col_index][0],agg[index]))
    middle['select'] = result
    return (middle ,replaced,True)
def add_select(middle,db_id,num = 1,agg = [''],distinct = [False]):
    table_idxs = middle['from']['table']
    col_ori = []
    col_aug = db_aug[db_id]
    for item in middle['select']:
        col_ori.append((item[0],item[2]))
    added = []
    step = 0
    result = []
    for index in range(num):
        flag = True
        while flag:
            if step >= 20:
                return (middle ,added, False)
            tab_index = random.randint(0,len(table_idxs)-1)
            col_idxs = col_aug[table_idxs[tab_index]]
            col_index = random.randint(0,len(col_idxs)-1)
            if num > 1:
                if col_index == 0:
                    step += 1
                    continue
            if col_index == 0:
                distinct[index] = False
            if (col_idxs[col_index][0],agg[index]) in col_ori:
                step += 1
                continue
            if agg[index] != '':
                if agg[index] in string_no_agg and col_idxs[col_index][2] == 'text':
                    step += 1
                    continue
                if agg[index] in value_no_agg and col_idxs[col_index][2] == 'number' :
                    step += 1
                    continue
                if agg[index] in time_no_agg and col_idxs[col_index][2] == 'time' :
                    step += 1
                    continue
            flag = False
            result.append([col_idxs[col_index][0],table_idxs[tab_index],agg[index],distinct[index]])
            added.append(col_idxs[col_index][1])
            col_ori.append((col_idxs[col_index][1],agg[index]))
    middle['select'].extend(result)   
    return (middle ,added, True)
def replace_count(middle,db_id,col = False):
    table_idxs = middle['from']['table']
    col_ori = []
    col_aug = db_aug[db_id]
    for item in middle['select']:
        col_ori.append((item[0],item[2]))
    added = []
    result = []
    index = 1
    flag = True
    step = 0
    
    while flag:
        if step >= 20:
            return (middle ,added, False)
        if col:
            tab_index = random.randint(0,len(table_idxs)-1)
            col_idxs = col_aug[table_idxs[tab_index]]
            col_index = random.randint(0,len(col_idxs)-1)
            if col_idxs[col_index][2] != 'text':
                step += 1
                continue
            if col_index == 0:
                step += 1
                continue
            if (col_idxs[col_index][0],'count') in col_ori:
                step += 1
                continue

            flag = False
            result.append([col_idxs[col_index][0],table_idxs[tab_index],'count',False])
            added.append(col_idxs[col_index][1])
        else:
            if (0,'count') in col_ori:
                return (middle,added,False)
            result.append([0,-1,'count',False])
            flag = False
    middle['select'].extend(result)
    return (middle ,added, True)
def replace_agg(middle,db_id,num = 1,theagg = ['']):
    table_idxs = middle['from']['table']
    col_ori = []
    col_db = db_dict[db_id]
    for item in middle['select']:
        if item[0] != 0:
            col_ori.append((item[0],item[2]))
    
    replaced = []
    result = copy.deepcopy(middle['select'])
    index = 0
    step = 0
    if len(col_ori) == 0 or len(col_ori) < num:
        return (middle,replaced,False)
    for index in range(num):
        flag = True
        while flag:
            if step >= 20:
                return (middle ,replaced, False)
            col_index = random.randint(0,len(result)-1)
            if (result[col_index][0],result[col_index][2]) not in col_ori:
                step += 1
                continue
            ttype = col_db['column_types'][result[col_index][0]]
            if ttype == 'text':
                agg = random.sample(['count'],1)[0]
            elif ttype == 'number':
                agg = random.sample(['sum','max','min','avg'],1)[0]
            else:
                agg = random.sample(['max','min'],1)[0]
            if (result[col_index][0],agg) in col_ori:
                step += 1
                continue
            if index <= len(theagg)-1 and theagg[0] != '':
                agg = theagg[index]
                index += 1
            flag = False
            result[col_index][2] = str(agg)
            replaced.append(agg)
    middle['select'] = result
    return (middle,replaced,True)
#[10, 1, '', 'one', '!=', '"Banking"', 'None']
def replace_where(middle,db_id,num = 1):
    table_idxs = middle['from']['table']
    where_ori = []
    col_db = db_dict[db_id]
    col_value = db_value[db_id]
    col_aug = db_aug[db_id]
    for item in middle['where']:
        where_ori.append((item[0],item[4]))
    result = []
    replaced = []
    index = 0
    step = 0
    for index in range(num):
        flag = True
        while flag:
            if step >= 20:
                return (middle ,replaced, False)
            tab_index = random.randint(0,len(table_idxs)-1)
            col_idxs = col_aug[table_idxs[tab_index]]
            col_index = random.randint(0,len(col_idxs)-1)
            if col_idxs[col_index][0] == 0:
                step += 1
                continue
            ttype = col_idxs[col_index][2]
            if ttype == 'text':
                op = random.sample(['=','!=','LIKE'],1)[0]
            elif ttype == 'number':
                op = random.sample(['=','!=','>','<','>=','<='],1)[0]
            else:
                op = random.sample(['=','!=','>','<','>=','<='],1)[0]
            value = ' '.join(random.sample(col_value[col_idxs[col_index][0]],1)[0])
            if (col_idxs[col_index][0],op) in where_ori:
                step += 1
                continue
            flag = False
            AND = 'one'
            if index > 0:
                AND = 'AND'
            if '"' not in value:
                value = '"' + value + '"'
            where_ori.append((col_idxs[col_index][0],op))
            result.append([col_idxs[col_index][0],table_idxs[tab_index],'',AND,op,value,'None'])
            replaced.append((col_idxs[col_index][1],op,value))
    middle['where'] = result
    return (middle,replaced,True)
def add_where(middle,db_id,num = 1):
    table_idxs = middle['from']['table']
    where_ori = []
    col_db = db_dict[db_id]
    col_value = db_value[db_id]
    col_aug = db_aug[db_id]
    
    if middle['where'] != None:
        for item in middle['where']:
            where_ori.append((item[0],item[4]))
        result = copy.deepcopy(middle['where'])
    else:
        result = []
    added = []
    index = len(where_ori)
    
    step = 0
    for _ in range(num):
        flag = True
        while flag:
            if step >= 20:
                return (middle ,added, False)
            tab_index = random.randint(0,len(table_idxs)-1)
            col_idxs = col_aug[table_idxs[tab_index]]
            col_index = random.randint(0,len(col_idxs)-1)
            
            if col_idxs[col_index][0] == 0:
                step += 1
                continue
            ttype = col_idxs[col_index][2]
            if ttype == 'text':
                op = random.sample(['=','!=','LIKE'],1)[0]
            elif ttype == 'number':
                op = random.sample(['=','!=','>','<','>=','<='],1)[0]
            else:
                op = random.sample(['=','!=','>','<','>=','<='],1)[0]
            value = ' '.join(random.sample(col_value[col_idxs[col_index][0]],1)[0])
            if (col_idxs[col_index][0],op) in where_ori:
                step += 1
                continue
            flag = False
            AND = 'one'
            if index > 0:
                AND = 'AND'
            index += 1
            if '"' not in value:
                value = '"' + value + '"'
            where_ori.append((col_idxs[col_index][0],op))
            result.append([col_idxs[col_index][0],table_idxs[tab_index],'',AND,op,value,'None'])
            added.append((col_idxs[col_index][1],op,value))
    middle['where'] = result
    return (middle,added,True)
#[10, 1, '', 'one', '!=', '"Banking"', 'None']
def replace_op(middle,db_id,num = 1):
    table_idxs = middle['from']['table']
    where_ori = []
    col_db = db_dict[db_id]
    col_value = db_value[db_id]
    col_aug = db_aug[db_id]
    replaced = []
    if middle['where'] == None:
        return (middle ,replaced, False)
    for item in middle['where']:
        where_ori.append((item[0],item[4]))
    if len(where_ori) < num:
        return (middle ,replaced, False)
    result = copy.deepcopy(middle['where'])
    index = 0
    step = 0
    for index in range(num):
        flag = True
        while flag:
            if step >= 20:
                return (middle ,replaced, False)
            col = random.sample(result,1)[0]
            example_index = result.index(col)
            col_index,tab_index = col[0],col[1]
            if col_index == 0:
                step += 1
                continue
            ttype = col_db['column_types'][col_index]
            if ttype == 'text':
                op = random.sample(['=','!=','LIKE'],1)[0]
            elif ttype == 'number':
                op = random.sample(['=','!=','>','<','>=','<='],1)[0]
            else:
                op = random.sample(['=','!=','>','<','>=','<='],1)[0]
            value = ' '.join(random.sample(col_value[col_index],1)[0])
            if (col_index,op) in where_ori:
                step += 1
                continue
            flag = False
            where_ori.append((col_index,op))
            result[example_index][4] = op
            if '"' not in value:
                value = '"' + value + '"'
            result[example_index][5] = value
            replaced.append((op,value))
    middle['where'] = result
    return (middle,replaced,True)
def add_orderby(middle,db_id,asc = 'ASC',limit = ''):
    table_idxs = middle['from']['table']
    where_ori = []
    col_db = db_dict[db_id]
    col_value = db_value[db_id]
    col_aug = db_aug[db_id]
    added = []
    if middle['orderBy'] != None:
        return (middle ,added, False)
    if len(middle['select']) == 1:
        if middle['select'][0][0] == 0 and middle['select'][0][2] == 'count':
            return (middle ,added, False)
    flag = True
    step = 0
    while flag:
        if step >= 20:
            return (middle ,added, False)
        tab_index = random.randint(0,len(table_idxs)-1)
        col_idxs = col_aug[table_idxs[tab_index]]
        col_index = random.randint(0,len(col_idxs)-1)
        if col_idxs[col_index][0] == 0:
            step += 1
            continue
        ttype = col_idxs[col_index][2]
        if ttype == 'text':
            step += 1
            continue
        
        flag = False
        middle['orderBy'] = [[col_idxs[col_index][0],table_idxs[tab_index],'',limit,asc]]
        added.append((col_idxs[col_index][1]))
    return (middle,added,True)
def add_having(middle,db_id,op,value):
    table_idxs = middle['from']['table']
    where_ori = []
    col_db = db_dict[db_id]
    col_value = db_value[db_id]
    col_aug = db_aug[db_id]
    added = []
    if middle['having'] != None:
        return (middle ,added, False)
    if middle['groupBy'] == None:
        return (middle ,added, False)
    flag = True
    step = 0
    while flag:
        if step >= 20:
            return (middle ,added, False)
        
        flag = False
        middle['having'] = [[0, -1, 'count', 'one', op, value, 'None']]
    return (middle,added,True)
def replace_asc(middle,db_id,asc = 'ASC'):
    table_idxs = middle['from']['table']
    where_ori = []
    col_db = db_dict[db_id]
    col_value = db_value[db_id]
    col_aug = db_aug[db_id]
    added = []
    if middle['orderBy'] == None:
        return (middle ,added, False)
    if middle['orderBy'][0][4] == asc:
        return (middle ,added, False)
    
    middle['orderBy'][0][4] = asc
    return (middle,added,True)
def add_groupby(middle,db_id):
    table_idxs = middle['from']['table']
    where_ori = []
    col_db = db_dict[db_id]
    col_value = db_value[db_id]
    col_aug = db_aug[db_id]
    added = []
    if middle['groupBy'] != None:
        return (middle ,added, False)
    flag = True
    step = 0
    while flag:
        if step >= 20:
            return (middle ,added, False)
        tab_index = random.randint(0,len(table_idxs)-1)
        col_idxs = col_aug[table_idxs[tab_index]]
        col_index = random.randint(0,len(col_idxs)-1)
        if col_idxs[col_index][0] == 0:
            step += 1
            continue
        ttype = col_idxs[col_index][2]
        if ttype != 'text':
            step += 1
            continue
        
        flag = False
        middle['groupBy'] = [[col_idxs[col_index][0],table_idxs[tab_index]]]
        added.append((col_idxs[col_index][1]))
    return (middle,added,True)

In [10]:
with open('preprocessed/context_ready.txt','r') as f:
    template_ori = f.readlines()
with open('preprocessed/alldata.json','r') as f:
    data_all = json.load(f)
# question_ori = [''] * len(data_all)
with open('preprocessed/final_generation.json','r') as f:
    question_ori = json.load(f)
template = []
index = 0
for item in template_ori:
    temp = {}
    line = item.strip()
    if len(line.split(' ||| ')) == 2:
        question,opt = line.split(' ||| ')
        constrain = None
    if len(line.split(' ||| ')) == 3:
        question,opt,constrain = line.split(' ||| ')
    temp['question'] = question
    temp['opt'] = eval(opt)
    if constrain != None:
        temp['constrain'] = eval(constrain)
    else:
        temp['constrain'] = None
    template.append(temp)
    index += 1

In [11]:
data_single = []
for index,item in enumerate(data_all):
    temp = {}
    temp['question'] = [question_ori[index]]
    if item['question'].count('[CLS]') == 0:
        temp['question'].append(item['question'])
    temp['middle'] = item['middle']
    temp['query'] = item['query']
    temp['db_id'] = item['db_id']
    data_single.append(temp)
data_db = {}
for item in data_single:
    if item['db_id'] not in data_db.keys():
        data_db[item['db_id']] = [item]
    else:
        data_db[item['db_id']].append(item)

In [12]:
def check_constrain(cur,template):
    if template['constrain'] != None:
        if 'compent' in template['constrain'].keys():
            for item in template['constrain']['compent']:
                if cur['middle'][item] == None:
                    return False
        if 'no_compent' in template['constrain'].keys():
            for item in template['constrain']['no_compent']:
                 if cur['middle'][item] != None:
                    return False
        if 'num_select' in template['constrain'].keys():
            if len(cur['middle']['select']) != template['constrain']['num_select']:
                return False
        if 'num_where' in template['constrain'].keys():
            if cur['middle']['where'] == None:
                return False
            if len(cur['middle']['where']) != template['constrain']['num_where']:
                return False

    return True

In [13]:
col_replace = ['{column0}','{column1}','{column2}']
agg_replace = ['{agg0}','{agg1}','{agg2}']
op_replace = ['{op0}','{op1}','{op2}']
value_replace = ['{value0}','{value1}','{value2}']
agg_trans = {'max':["maximum","maximum number","maximum amount","the largest","the highest"],\
 'min':["minimum","minimum number","minimum number","the smallest","the lowest",],\
 'count':["the number"],\
 'avg':["the average","the average number","the mean"],\
 'sum':["the total number","total count","sum of","in total","the number of all","the total amount"]}
op_trans = {
    "=": ["is", "was", "are", "equal"],
    ">": ["more than", "higher than", "above", "bigger than", "over", "above"],
    "<": ["less than", "lower than", "below", "small than", "under", "below"],
    ">=": ["no less than", "no lower than", "no below", "no small than", 
     "no under", "at least", "or more", "no below", "not less than", "not lower than", "not below",
      "not small than",  "not before", "not under", "not below"],
    "<=": ["no more than", "no higher than", "no above", "no bigger than",
     "no over", "no above", "at most", "or less", "not more than", "not higher than", "not above",
      "not bigger than",  "not after", "not over", "not above"],
    "!=": ["does not equal to", "is not", "that are not", "were not", "that was not", "is non", "excluding", "not including",
      "except for", "ignore", "outside", "other than"],
    "LIKE": ["has the substring", "contains the word", "contains", "containing", "have substring", "starts with", "have the letter", "includes the substring"]
}
def generation_one(cur,template):
    one = {}
    one['db_id'] = cur['db_id']
    flag = False
    col_index = 0
    agg_index = 0
    op_index = 0
    value_index = 0
    db_id = cur['db_id']
    one['question'] = template['question']
    if check_constrain(cur,template):
        middle = cur['middle']
        
        if 'replace_select' in template['opt'].keys():
            num = 1
            agg = ['']
            distinct = False
            if template['opt']['replace_select'] != None:
                num = len(template['opt']['replace_select'])
                agg = template['opt']['replace_select']
                
                distinct = [False] * num
            middle,col,isok = replace_select(middle,db_id,num ,agg,distinct)

            if isok == False:
                return (one,False)
            for idx in range(num):
                one['question'] = one['question'].replace(col_replace[col_index],col[idx])
                col_index += 1
                    
        if 'add_select' in template['opt'].keys():
            num = 1
            agg = ['']
            distinct = False
            if template['opt']['add_select'] != None:
                num = len(template['opt']['add_select'])
                agg = template['opt']['add_select']
                
                distinct = [False] * num
            middle,col,isok = add_select(middle,db_id,num ,agg,distinct)

            if isok == False:
                return (one,False)
            for idx in range(num):
                one['question'] = one['question'].replace(col_replace[col_index],col[idx])
                col_index += 1

        if 'replace_agg' in template['opt'].keys():
            num = 1
            if template['opt']['replace_agg'] != None:
                #todo
                pass
                
            middle,col,isok = replace_agg(middle,db_id,num)
            if isok == False:
                return (one,False)
            for _ in range(num):
                one['question'] = one['question'].replace(agg_replace[agg_index],random.sample(agg_trans[col[agg_index]],1)[0])
                agg_index += 1
        
        if 'replace_count' in template['opt'].keys():
            needcol = False
            num = 1
            if template['opt']['replace_count'] != None:
                needcol = True
            middle,col,isok = replace_count(middle,db_id,needcol)
            if isok == False:
                return (one,False)
            if needcol:
                for idx in range(num):
                    one['question'] = one['question'].replace(col_replace[col_index],col[idx])
                    col_index += 1

                
        if 'replace_where' in template['opt'].keys():
            num = 1
            agg = ['']
            distinct = False
            if template['opt']['replace_where'] != None:
                #todo
                pass
 
            middle,col,isok = replace_where(middle,db_id,num)
            if isok == False:
                return (one,False)
            for _ in range(num):
                one['question'] = one['question'].replace(col_replace[col_index],col[0][0])
                one['question'] = one['question'].replace(op_replace[col_index],random.sample(op_trans[col[0][1]],1)[0])
                one['question'] = one['question'].replace(value_replace[value_index],col[0][2])
                col_index += 1
                op_index += 1
                value_index += 1
                
        if 'add_where' in template['opt'].keys():
            num = 1
            if template['opt']['add_where'] != None:
                #todo
                pass
 
            middle,col,isok = add_where(middle,db_id,num)
            if isok == False:
                return (one,False)
            for _ in range(num):
                one['question'] = one['question'].replace(col_replace[col_index],col[0][0])
                one['question'] = one['question'].replace(op_replace[col_index],random.sample(op_trans[col[0][1]],1)[0])
                one['question'] = one['question'].replace(value_replace[value_index],col[0][2])
                col_index += 1
                op_index += 1
                value_index += 1
        
        if 'replace_op' in template['opt'].keys():
            num = 1
            if template['opt']['replace_op'] != None:
                #todo
                pass
 
            middle,col,isok = replace_op(middle,db_id,num)
            if isok == False:
                return (one,False)
            for _ in range(num):
                one['question'] = one['question'].replace(op_replace[col_index],random.sample(op_trans[col[0][0]],1)[0])
                one['question'] = one['question'].replace(value_replace[value_index],col[0][1])
                op_index += 1
                value_index += 1
       
        
        if 'add_orderby' in template['opt'].keys():
            asc = 'ASC'
            limit = ''
            if template['opt']['add_orderby'] != None:
                asc = template['opt']['add_orderby'][0][0]
                limit = template['opt']['add_orderby'][0][1]
 
            middle,col,isok = add_orderby(middle,db_id,asc,limit)
            if isok == False:
                return (one,False)
            one['question'] = one['question'].replace(col_replace[col_index],col[0])
            col_index += 1
        
        
        
        if 'replace_asc' in template['opt'].keys():
            asc = 'ASC'
            limit = ''
            if template['opt']['replace_asc'] != None:
                asc = template['opt']['replace_asc']

            middle,col,isok = replace_asc(middle,db_id,asc)
            if isok == False:
                return (one,False)
            
        if 'add_groupby' in template['opt'].keys():
            if template['opt']['add_groupby'] != None:
                #todo
                pass
 
            middle,col,isok = add_groupby(middle,db_id)
            if isok == False:
                return (one,False)
            one['question'] = one['question'].replace(col_replace[col_index],col[0])
            col_index += 1
        
        if 'add_having' in template['opt'].keys():
            asc = 'ASC'
            limit = ''
            if template['opt']['add_having'] != None:
                op = template['opt']['add_having'][0][0]
                value = template['opt']['add_having'][0][1]
 
            middle,col,isok = add_having(middle,db_id,op,value)
            if isok == False:
                return (one,False)
            
    else:
        return (one,False)
#     one['middle'] = middle
    one['middle'] = middle
    return (one,True)

In [14]:
def generation_multi(single,num,turn = 5):
    data_multi = []
#     first = random.sample(single,1)[0]
    first = copy.deepcopy(single)
    temp = copy.deepcopy(first)
    temp['relate'] = 0
    temp['question'] = random.sample(temp['question'],1)[0]
    data_multi.append(temp)
    index = 1
    index1 = 1
    total = 0
    his = [first['query']]
    relate = 1
    therelate = 0
    while total < turn:
        if random.random() > 0.4 and therelate < 3:
            therelate += 1
            flag = True
            while flag:
                if  index > num:
                    return (data_multi,False)
                cur_temp = random.sample(template,1)[0]
                one,isok = generation_one(first,cur_temp)
                
                if not isok:
                    index += 1
                    continue
                one['query'] = translate_sql(one['middle'],0,one['db_id'],db_reverse)
                if one['query'] in his:
#                 print(get_struct_similarity(one['query'], his))
                # if 1 in get_struct_similarity(one['query'], his):
                    # print('chongfu')
                    index += 1
                    continue

#                 print(one)
                temp = copy.deepcopy(one)
                temp['relate'] = relate
                data_multi.append(temp)
                his.append(one['query'])
#                 print(total)
                total += 1
                flag = False
                if random.random() > 0.05:
                    first = one
                    relate = 1
                else:
                    relate = relate + 1
        else:
            therelate = 0
            flag = True
            while flag:
                if  index1 > num:
                    flag = False
                second = copy.deepcopy(random.sample(data_db[first['db_id']],1)[0])
#                 print(get_struct_similarity(one['query'], his))
                # if second['query'] == first['query']:
                if second['query'] in his:
                    # print('chongfu')
                    index1 += 1
                    continue
                temp = copy.deepcopy(second)
                temp['relate'] = 0
                temp['question'] = random.sample(temp['question'],1)[0]
                data_multi.append(temp)
                his.append(second['query'])
                total += 1
                flag = False
                if random.random() > 0.05:
                    first = second
                    relate = 1
                else:
                    relate = relate + 1
    return (data_multi,True)

In [15]:
from tqdm import tqdm
session = []
for _ in range(10):
    for item in tqdm(data_single):
        second = copy.deepcopy(item)
        try:
            one,_ = generation_multi(second,1000,4)
        except:
    #         print(item)
            pass
        session.append(one)

100%|██████████| 15281/15281 [00:11<00:00, 1383.69it/s]


In [16]:
with open('raw_data/tables.json','r') as f:
    table = json.load(f)
db = {}
for item in table:
    temp = {}
    temp['column_names'] = item['column_names']
    temp['column_types'] = item['column_types']
    temp['table_names'] = item['table_names']
    temp['column_names_original'] = item['column_names_original']
    temp['table_names_original'] = item['table_names_original']
    
    table2columns = [[] for _ in range(len(item['table_names']))] # from table id to column ids list
    for col_id, col in enumerate(item['column_names']):
        if col_id == 0: continue
        table2columns[col[0]].append(col_id)
    for col,typ in zip(item['column_names'],item['column_types']):
        col.append(typ)
    temp['table2columns'] = table2columns
    db[item['db_id']] = temp
db_reverse = {}
db_c_ori2pre = {}
db_t_ori2pre = {}
for k,v in db.items():
    db_reverse[k] = {}
    column = []
    table = []
    db_c_ori2pre[k] = {}
    db_t_ori2pre[k] = {}
    for o,c in zip(v['column_names'],v['column_names_original']):
        column.append([o[0],o[1],c[1]])
        db_c_ori2pre[k][c[1]] = o[1]
    db_reverse[k]['column'] = column
    for o,c in zip(v['table_names'],v['table_names_original']):
        table.append([o,c])
        db_t_ori2pre[k][c] = o
    db_reverse[k]['table'] = table

In [18]:
from grakel.kernels import WeisfeilerLehman, VertexHistogram
import sqlparse
from sqlparse.tokens import Token

def similarity_compute_(s1,s2):
    s1_ = s1.split(" ")
    s2_ = s2.split(" ")
    l_min = min(len(s1_),len(s2_))
    l_max = max(len(s1_),len(s2_))
    return sum([1 if s1_[i] in s2_ else 0 for i in range(l_min)]) / l_max


def similarity_compute(l1,l2):
    assert len(l1)==len(l2)
    l = sum([0 if l1[i]=='' and l2[i]=='' else 1 for i in range(len(l1))])
    return sum([similarity_compute_(l1[i],l2[i]) if l1[i]!='' and l2[i]!='' else 0 for i in range(len(l1))]) / l


def get_wl_score(samples_list):
    gk = WeisfeilerLehman(n_iter=5, base_graph_kernel=VertexHistogram, normalize=True)
    K_samples = gk.fit_transform(samples_list)
    return K_samples[0][1:]

def _pprint_tree(sql_node_label, sql_depth_label, sql_graph, tokens, max_depth=None, depth=0, f=None, _pre=''):  # Deepth First Search
    """Pretty-print the object tree."""
    token_count = len(tokens)
    for idx, token in enumerate(tokens):
        if token.ttype in [Token.Text.Whitespace, Token.Name]: #, Token.Punctuation, Token.Name, Token.Wildcard]:
            continue
        if token.value not in sql_node_label.keys():
            sql_node_label[token.value] = len(sql_node_label.items())
        sql_graph.append((sql_node_label[sql_depth_label[depth-1]], sql_node_label[token.value]))
        sql_depth_label[depth] = token.value
        cls = token._get_repr_name()
        value = token._get_repr_value()

        last = idx == (token_count - 1)
        pre = '`- ' if last else '|- '

        q = '"' if value.startswith("'") and value.endswith("'") else "'"
        # print("{_pre}{pre}{idx} {cls} {q}{value}{q}"
        #         .format(**locals()), file=f)
        # print("{_pre}{pre}{depth} {value}"
        #         .format(**locals()))
        if token.is_group and (max_depth is None or depth < max_depth):
            parent_pre = '   ' if last else '|  '
            _pprint_tree(sql_node_label, sql_depth_label, sql_graph, token.tokens,max_depth, depth + 1, f, _pre + parent_pre)

    return sql_node_label, sql_depth_label, sql_graph
    
def get_struct_similarity(curt_sql, hist_sql):
    sql_list = [curt_sql] + hist_sql
    sql_node_label = {'root':0}
    samples_list_ = []
    samples_list = []
    for sl in sql_list:
        sample = []
        sql_depth_label = {-1:'root'}
        sql_graph = []
        sql_node_label, sql_depth_label, sql_graph = _pprint_tree(sql_node_label, sql_depth_label, sql_graph, sqlparse.parse(sl)[0].tokens)
        sample.append(set(sql_graph))

        sql_edge_label = dict([(v,0) for v in set(sql_graph)])
        sample.append(sql_edge_label)
        samples_list_.append(sample)
    for sl in samples_list_:
        sl.insert(1,  dict([(idx,sql_node_label[v]) for idx, v in enumerate(sql_node_label.keys())]))
        # sl.insert(1,  dict([(idx, 0) for idx, v in enumerate(sql_node_label.keys())]))
        samples_list.append(sl)
    # print(sql_depth_label)
    # print(samples_list)
    # print(sql_node_label)
    wl_score = get_wl_score(samples_list)
    return list(wl_score)


# score = get_struct_similarity(sql, hist_sql)
# print(score)
def compute_content(cur_label,his_label):
    result = []
    for item in his_label:
        result.append(similarity_compute(cur_label,item))
    return result

In [19]:
schemas, db_names, tables = get_schemas_from_json('raw_data/tables.json')
from tqdm import tqdm
data_final = []
for idx,item in enumerate(tqdm(session)):
    last = []
    last_sql = []
    last_label = []
    last_clabel = []
    last_tlabel = []
    relate = []
    theturn = []
    flag = True
    for turn in item:
        if flag == False:
            continue
        temp = {}
        last.append(turn['question'])
        last_sql.append(turn['query'])
        relate.append(turn['relate'])
        
        db_id = turn['db_id']
        schema = schemas[db_id]
        table = tables[db_id]
        schema = Schema(schema, table)
        try:
            sql_label = get_sql(schema, turn['query'])
            col_label = get_label(sql_label,len(table['column_names_original']))
            table_label = get_table_label(sql_label,len(table['table_names']))

            label = col_label + table_label
            last_label.append(label)
            last_clabel.append(col_label)
            last_tlabel.append(table_label)

            temp['col_label'] = last_clabel[::-1][:5]
            temp['table_label'] = last_tlabel[::-1][:5]
            temp['sql'] = sql_label
            temp['db_id'] = turn['db_id']
            temp['query'] = turn['query']
            temp['question'] = ' [CLS] '.join(last[::-1][:5])
            temp['struct'] = []
            for sql in last_sql[::-1][:5]:
                temp['struct'].append(get_struct_similarity(temp['query'], [sql])[0])

            temp['content'] = compute_content(col_label, last_clabel[::-1][:5])
            temp['sim'] = [(temp['struct'][i]+temp['content'][i])/2 for i in range(len(temp['content']))]
            temp['relate'] = relate[::-1][:5]
            assert len(temp['content']) == temp['question'].count('[CLS]')+1
            theturn.append(temp)
        except:
            flag = False
            print(idx)
    if flag:
        data_final.append(theturn)

  0%|          | 243/152810 [00:17<2:32:37, 16.66it/s]

239


  1%|          | 1371/152810 [01:41<1:54:50, 21.98it/s]

1367


  1%|          | 1395/152810 [01:43<2:08:13, 19.68it/s]

...

100%|█████████▉| 152252/152810 [3:07:27<00:28, 19.85it/s]

152249


100%|█████████▉| 152596/152810 [3:07:50<00:15, 13.88it/s]

152593


100%|██████████| 152810/152810 [3:08:05<00:00, 13.54it/s]


In [20]:
total = 0
temp = {}
final = []
for turn in data_final:
    flag = True
    for idx,item in enumerate(turn):
        if item['question'] in temp.keys() and idx != 0:
            flag = False
            continue
        temp[item['question']] = total
        total += 1
    if flag:
        final.append(turn)

In [21]:
import numpy as np 
for item in final:
    temp = []
    for iitem in item:
        temp.append(iitem['sim'][::-1][:-1] + [0]*(5-len(iitem['sim'][:-1])))
    temp = (np.array(temp).T + np.array(temp) + np.eye(5)).tolist()
    for idx,iitem in enumerate(item):
        iitem['final_sim'] = temp[idx]

In [28]:
label_dict = {'':0}
for item in final:
    for iitem in item:
        for col in iitem['col_label'][0]:
            if col not in label_dict.keys():
                label_dict[col] = 1
            else:
                label_dict[col] += 1
count = 0
thelabel = []
finallabel = {}
convert = {}
index = 0
label_dict = dict(sorted(label_dict.items(), key=lambda x: x[1], reverse=True))
for k,v in label_dict.items():
    count += 1
    thelabel.append(k)
    finallabel[k] = v
    convert[k] = index
    index += 1
    if count == 383:
        break
theall = []
for item in final:
    flag = True
    for iitem in item:
        for col in iitem['col_label'][0]:
            if col not in thelabel:
                flag = False
    if flag:
        theall.append(item)

In [29]:
len(theall)

64609

In [32]:
count

323

In [33]:
convert

{'': 0,
 'SELECT COUNT': 1,
 'GROUP_BY': 2,
 'SELECT': 3,
 'GROUP_BY ORDER_BY ASC LIMIT': 4,
 'GROUP_BY ORDER_BY DESC LIMIT': 5,
 'WHERE OR =': 6,
 'ORDER_BY COUNT DESC LIMIT': 7,
 'WHERE =': 8,
 'SELECT WHERE >=': 9,
 'SELECT COUNT HAVING COUNT >=': 10,
 'SELECT GROUP_BY': 11,
 'SELECT GROUP_BY ORDER_BY DESC LIMIT': 12,
 'SELECT WHERE =': 13,
 'SELECT AVG': 14,
 'WHERE >=': 15,
 'HAVING COUNT >': 16,
 'SELECT COUNT GROUP_BY': 17,
 'SELECT SUM GROUP_BY': 18,
 'SELECT MIN': 19,
 'SELECT MIN SELECT': 20,
 'SELECT SELECT COUNT': 21,
 'WHERE LIKE': 22,
 'SELECT EXCEPT SELECT': 23,
 'EXCEPT WHERE =': 24,
 'ORDER_BY DESC LIMIT': 25,
 'SELECT ORDER_BY DESC': 26,
 'SELECT ORDER_BY DESC LIMIT': 27,
 'WHERE IN': 28,
 'OP_SEL SELECT': 29,
 'WHERE !=': 30,
 'SELECT ORDER_BY COUNT DESC LIMIT': 31,
 'HAVING COUNT >=': 32,
 'SELECT COUNT ORDER_BY DESC LIMIT': 33,
 'EXCEPT SELECT': 34,
 'SELECT COUNT ORDER_BY COUNT DESC LIMIT': 35,
 'SELECT SELECT': 36,
 'ORDER_BY ASC LIMIT': 37,
 'SELECT ORDER_BY ASC

In [41]:
def get_position_ids(ex, shuffle=True):
    # cluster columns with their corresponding table and randomly shuffle tables and columns
    # [CLS] q1 q2 ... [SEP] * t1 c1 c2 c3 t2 c4 c5 ... [SEP]
    db, table_word_len, column_word_len = ex['db'], ex['table_word_len'], ex['column_word_len']
    table_num, column_num = len(db['table_names']), len(db['column_names'])
    question_position_id = list(range(len(ex['question_id'])))
    start = len(question_position_id)
    table_position_id, column_position_id = [None] * table_num, [None] * column_num
    column_position_id[0] = list(range(start, start + column_word_len[0]))
    start += column_word_len[0] # special symbol * first
    table_idxs = list(range(table_num))
    if shuffle:
        random.shuffle(table_idxs)
    for idx in table_idxs:
        col_idxs = db['table2columns'][idx]
        table_position_id[idx] = list(range(start, start + table_word_len[idx]))
        start += table_word_len[idx]
        if shuffle:
            random.shuffle(col_idxs)
        for col_id in col_idxs:
            column_position_id[col_id] = list(range(start, start + column_word_len[col_id]))
            start += column_word_len[col_id]
    position_id = question_position_id + list(chain.from_iterable(table_position_id)) + \
        list(chain.from_iterable(column_position_id)) + [start]
    assert len(position_id) == len(ex['input_id'])
    return position_id

In [43]:
from transformers import AutoTokenizer
from itertools import chain
import numpy as np
t = AutoTokenizer.from_pretrained("google/electra-large-discriminator")
def get_input(item, db, t):
    s = re.sub('([.,!?()])', r' \1 ', item['question'])
    s = re.sub('\s{2,}', ' ', s)
    input_token = '[CLS]'
    question = [q.lower() for q in s.split()]
    question = ['[CLS]' if q == '[cls]' else q for q in question]
    sim_label = item['final_sim']
    sim_mask = [True if item != -1 else False for item in item['final_sim']]
    rtd_label = []
    ssf_label = []
    context_label = []
    question_id = [t.cls_token_id] # map token to id
    question_mask_plm = []
    question_subword_len = []
    input_token += ' ' + ' '.join(question)
    for w in question:
        toks = t.convert_tokens_to_ids(t.tokenize(w))
        question_id.extend(toks)
        question_subword_len.append(len(toks))
    question_mask_plm = [0] + [1] * (len(question_id) - 1) + [0]


    question_id.append(t.sep_token_id)
    input_token += ' [SEP]'
    rtd_label = [0]*len(question_id)

    table = [['table'] + t.lower().split() for t in db['table_names']]
    table_id, table_mask_plm, table_subword_len = [], [], []
    table_word_len = []
    for tab_ids, s in enumerate(table):
        l = 0
        for w in s:
            input_token += ' ' + w
            toks = t.convert_tokens_to_ids(t.tokenize(w))
            table_id.extend(toks)
            table_subword_len.append(len(toks))
            l += len(toks)
        if len(item['table_label'][0][tab_ids]) == 0:
            rtd_label.extend([1]*l)
        else:
            rtd_label.extend([0]*l)
        table_word_len.append(l)
    table_mask_plm = [1] * len(table_id)

    if len(item['col_label']) == 1:
        column = [[db['column_types'][idx].lower()] + c.lower().split() for idx, (_, c, _) in enumerate(db['column_names'])]
    else:
        column = [[db['column_types'][idx].lower()] + c.lower().split() + item['col_label'][1][idx].split() for idx, (_, c, _) in enumerate(db['column_names'])]
    col_index = 0
    column_id, column_mask_plm, column_subword_len = [], [], []
    column_word_len = []
    for col_ids, s in enumerate(column):
        l = 0
        ssf_label.append(convert[item['col_label'][0][col_ids]])
        for w in s:
            input_token += ' ' + w
            toks = t.convert_tokens_to_ids(t.tokenize(w))
            column_id.extend(toks)
            column_subword_len.append(len(toks))
            l += len(toks)
        if len(item['col_label'][0][col_ids]) == 0:
            rtd_label.extend([1]*l)
        else:
            rtd_label.extend([0]*l)
        column_word_len.append(l)

    rtd_label.append(0)
    column_mask_plm = [1] * len(column_id) + [0]
    
    input_token += ' [SEP]'
    column_id.append(t.sep_token_id)
    
    question_mask_plm = question_mask_plm + [0] * (len(table_id) + len(column_id))
    table_mask_plm = [0] * len(question_id) + table_mask_plm + [0] * len(column_id)
    column_mask_plm = [0] * (len(question_id) + len(table_id)) + column_mask_plm

    input_id = question_id + table_id + column_id
    
   
    
    assert len(input_id) == len(question_mask_plm) == len(table_mask_plm) == len(column_mask_plm) == len(rtd_label)
    assert len(ssf_label) == len(item['col_label'][0])
    assert len(sim_label) == 5
    result = {}
    result['db_id'] = item['db_id']
    result['input_id'] = input_id
    result['query'] = item['query']
    result['sql'] = item['sql']
    result['content'] = item['content']
    result['db_id'] = item['db_id']
    result['sim_label'] = sim_label
    result['sim_mask'] = sim_mask
    result['ssf_label'] = ssf_label
    result['question_mask_plm'] = question_mask_plm
    result['rtd_label'] = rtd_label
    temp = {}
    temp['db'] = db
    temp['table_word_len'] = table_word_len
    temp['column_word_len'] = column_word_len
    temp['question_id'] = question_id
    temp['input_id'] = input_id

    result['column_mask_plm'] = column_mask_plm
    result['column_word_len'] = column_word_len
    result['position_ids'] = get_position_ids(temp, shuffle=True)
    result['input_token'] = input_token
    result['db_id'] = item['db_id']
    return result

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/668 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [44]:
from tqdm import tqdm
alltask = []
for item in tqdm(theall):
    temp = []
    for iitem in item:
        one = get_input(iitem,db[iitem['db_id']],t)
        temp.append(one)
    alltask.append(temp)

100%|██████████| 64609/64609 [46:16<00:00, 23.27it/s]  


In [45]:
from tqdm import tqdm
with open('final_data/electra_all.json','w') as f:
    json.dump(alltask,f)

In [46]:
final = []
for item in alltask:
    flag = True
    for iitem in item:
        if len(iitem['input_id']) > 256:
            flag = False
    if flag:
        final.append(item)

In [47]:
db_data = {}
for item in final:
    if item[0]['db_id'] not in db_data.keys():
        db_data[item[0]['db_id']] = [item]
    else:
        db_data[item[0]['db_id']].append(item)
keys = []
for key in db_data.keys():
    keys.append(key)
weight = [len(v) for k, v in db_data.items()]

In [48]:
final = []
for i in range(12000):
    thekeys = random.choices(keys,weights=weight,k=8)
    batch = []
    for key in thekeys:
        temp = random.sample(db_data[key],1)[0]
        batch.append(temp)
    for item in batch:
        for iitem in item:
            final.append(iitem)

In [49]:
from tqdm import tqdm
with open('final_data/alltask_final.txt','w') as f:
    for item in tqdm(final):
        f.write(json.dumps(item))
        f.write('\n')

100%|██████████| 480000/480000 [00:45<00:00, 10586.93it/s]


In [50]:
len(final)

480000